In [2]:
import os
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf

# === SETTINGS ===
base_dir = r"C:\Users\zezom\PycharmProjects\HorusEye"
dl_ready_dir = os.path.join(base_dir, "Data", "Processed", "DL_ready")
test_csv = os.path.join(base_dir, "test.csv")

# === LOAD MODELS & ENCODERS ===
mi_model = tf.keras.models.load_model(os.path.join(base_dir, "mi_model.keras"))
ssvep_model = tf.keras.models.load_model(os.path.join(base_dir, "ssvep_model.keras"))

with open(os.path.join(base_dir, "mi_label_encoder.pkl"), "rb") as f:
    mi_label_encoder = pickle.load(f)
with open(os.path.join(base_dir, "ssvep_label_encoder.pkl"), "rb") as f:
    ssvep_label_encoder = pickle.load(f)

# === LOAD TEST DATA ===
test_df = pd.read_csv(test_csv)
test_df.columns = test_df.columns.str.strip()  # Remove whitespace

# === PREDICTION STORAGE ===
results = []

for idx, row in test_df.iterrows():
    subject = str(row['subject_id']).strip()
    session = str(row['trial_session']).strip()
    trial_num = int(row['trial'])  # 1-based
    paradigm = row['paradigm'].strip() if 'paradigm' in row else None
    sample_id = row['sample_id'] if 'sample_id' in row else idx  # Use 'sample_id' if available

    npy_path = os.path.join(dl_ready_dir, f"{subject}_{session}_EEGdata_preprocessed_DLready.npy")
    if not os.path.exists(npy_path):
        print(f"Missing EEG file for test row {idx}: {npy_path}")
        results.append({"sample_id": sample_id, "label": "Unknown"})
        continue

    epochs = np.load(npy_path)  # (n_trials, n_channels, n_samples)
    if trial_num - 1 >= epochs.shape[0]:
        print(f"Trial number {trial_num} out of range in {npy_path}")
        results.append({"sample_id": sample_id, "label": "Unknown"})
        continue

    x = epochs[trial_num - 1]  # (n_channels, n_samples)
    x = np.transpose(x, (1, 0))  # (n_samples, n_channels) for Conv1D/LSTM
    x = np.expand_dims(x, axis=0)  # Add batch dimension: (1, n_samples, n_channels)

    if paradigm is not None:
        is_ssvep = (paradigm.upper() == 'SSVEP')
    else:
        # If no 'paradigm' column, infer from label or filename
        is_ssvep = x.shape[1] > 4000  # <-- adjust this logic to your data!

    if is_ssvep:
        pred = ssvep_model.predict(x, verbose=0)
        label = ssvep_label_encoder.inverse_transform([np.argmax(pred)])[0]
    else:
        pred = mi_model.predict(x, verbose=0)
        label = mi_label_encoder.inverse_transform([np.argmax(pred)])[0]

    results.append({
        "sample_id": sample_id,
        "label": label
    })

# === OUTPUT SUBMISSION ===
submission = pd.DataFrame(results)
# If competition expects exactly two columns: sample_id,label
submission = submission[['sample_id', 'label']]
out_path = os.path.join(base_dir, "submission.csv")
submission.to_csv(out_path, index=False)
print(f"Submission file saved to: {out_path}")


C:\Users\zezom\miniconda3\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 21 variables whereas the saved optimizer has 40 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Missing EEG file for test row 0: C:\Users\zezom\PycharmProjects\HorusEye\Data\Processed\DL_ready\S36_1_EEGdata_preprocessed_DLready.npy
Missing EEG file for test row 1: C:\Users\zezom\PycharmProjects\HorusEye\Data\Processed\DL_ready\S36_1_EEGdata_preprocessed_DLready.npy
Missing EEG file for test row 2: C:\Users\zezom\PycharmProjects\HorusEye\Data\Processed\DL_ready\S36_1_EEGdata_preprocessed_DLready.npy
Missing EEG file for test row 3: C:\Users\zezom\PycharmProjects\HorusEye\Data\Processed\DL_ready\S36_1_EEGdata_preprocessed_DLready.npy
Missing EEG file for test row 4: C:\Users\zezom\PycharmProjects\HorusEye\Data\Processed\DL_ready\S36_1_EEGdata_preprocessed_DLready.npy
Missing EEG file for test row 5: C:\Users\zezom\PycharmProjects\HorusEye\Data\Processed\DL_ready\S36_1_EEGdata_preprocessed_DLready.npy
Missing EEG file for test row 6: C:\Users\zezom\PycharmProjects\HorusEye\Data\Processed\DL_ready\S36_1_EEGdata_preprocessed_DLready.npy
Missing EEG file for test row 7: C:\Users\zezom\